In [1]:
date = 20220930


import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

AUTO_NA = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\04_AITAB_Financing\03_Newly_Approved\\'+str(date)+'\Listing approved_'+str(date)+'.xlsx', sheet_name='Listing ')

location_application = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(date)+"\LOS_HPS_APPLICATION_DETAILS"+str(date)+".txt"

application = pd.read_csv(location_application,sep = "\t", header = 0)

application.columns = application.columns.str.replace(" ", "_")

AUTO_NA['Goods_Status'] = AUTO_NA['Goods_Status'].str.strip()

AUTO_NA.Scoring.fillna(0, inplace=True)

def RISK_GRADE(Goods_Status,Scoring):
    if Goods_Status == "U":
        if Scoring == 0:
            return "09.UNRATED"
        elif Scoring > 0 and Scoring < 191:
            return "08.R8"
        elif Scoring >= 191 and Scoring < 212:
            return "07.R7"
        elif Scoring >= 212 and Scoring < 230:
            return "06.R6"
        elif Scoring >= 230 and Scoring < 245:
            return "05.R5"
        elif Scoring >= 245 and Scoring < 259:
            return "04.R4"
        elif Scoring >= 259 and Scoring < 275:
            return "03.R3"
        elif Scoring >= 275 and Scoring < 293:
            return "02.R2"
        elif Scoring >= 293:
            return "01.R1"
    elif Scoring == 0:
        return "09.UNRATED"
    elif Scoring > 0 and Scoring < 196:
        return "08.R8"
    elif Scoring >= 196 and Scoring < 216:
        return "07.R7"
    elif Scoring >= 216 and Scoring < 233:
        return "06.R6"
    elif Scoring >= 233 and Scoring < 249:
        return "05.R5"
    elif Scoring >= 249 and Scoring < 266:
        return "04.R4"
    elif Scoring >= 266 and Scoring < 285:
        return "03.R3"
    elif Scoring >= 285 and Scoring < 310:
        return "02.R2"
    elif Scoring >= 310:
        return "01.R1"
AUTO_NA['RISK_GRADE'] = AUTO_NA.apply(lambda x: RISK_GRADE(x['Goods_Status'], x['Scoring']), axis=1)

AUTO_NA['Hps_Description'] = AUTO_NA['Hps_Description'].str.strip()

AUTO_NA.loc[AUTO_NA.Hps_Description.isin(["UP TO RM999","RM1,000 TO RM1,999","RM2,000 TO RM2,999","UP TO RM999"]),'B_INCOME'] = "01.Up to RM3000"
AUTO_NA.loc[AUTO_NA.Hps_Description.isin(["RM3,000 TO RM3,999","RM4,000 TO RM4,999"]),'B_INCOME'] = "02.>Rm3000 to Rm5000"
AUTO_NA.loc[AUTO_NA.Hps_Description.isin(["RM5,000 TO RM5,999","RM6,000 TO RM6,999","RM7,000 TO RM7,999"]),'B_INCOME'] = "03.>Rm5000 to Rm8000"
AUTO_NA.loc[AUTO_NA.Hps_Description.isin(["RM10,000 TO RM10,999","RM11,000 TO RM14,999","RM8,000 TO RM8,999","RM9,000 TO RM9,999"]),'B_INCOME'] = "04.>Rm8000 to Rm15000"
AUTO_NA.loc[AUTO_NA.Hps_Description.isin(["RM15,000 TO RM20,000"]),'B_INCOME'] = "05.Rm15,000 TO Rm20,000"
AUTO_NA.loc[AUTO_NA.Hps_Description.isin(["ABOVE RM20,000"]),'B_INCOME'] = "06.Above Rm20,000"
AUTO_NA.loc[AUTO_NA.Hps_Description.isin(["Not Available"]),'B_INCOME'] = "07.Not Available"

def B_MOF(MOF):
    if MOF < 70:
        return "01.<70"
    elif MOF >= 70 and MOF < 80:
        return "02.70 TO <80"
    elif MOF >= 80 and MOF < 85:
        return "03.80 TO <85"
    elif MOF >= 85:
        return "04.>=85%"
    else:
        return "05.NA"
AUTO_NA['B_MOF'] = AUTO_NA.MOF.apply(B_MOF)

def B_TENURE(Tenor):
    if Tenor < 48:
        return "01.<=3 yrs"
    elif Tenor >= 48 and Tenor < 72:
        return "02.4-5 yrs"
    elif Tenor >= 72 and Tenor < 96:
        return "03.6-7 yrs"
    elif Tenor >= 96:
        return "04.>=8 yrs"
AUTO_NA['B_TENURE'] = AUTO_NA.Tenor.apply(B_TENURE)

AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["PERODUA"]), 'Vehicle_Make2'] = "01.PERODUA"
AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["TOYOTA"]), 'Vehicle_Make2'] = "02.TOYOTA"
AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["PROTON"]), 'Vehicle_Make2'] = "03.PROTON"
AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["HONDA","HONDA MOTOCYCLE"]), 'Vehicle_Make2'] = "04.HONDA"
AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["NISSAN"]), 'Vehicle_Make2'] = "05.NISSAN"
AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["M/BENZ"]), 'Vehicle_Make2'] = "06.M/BENZ"
AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["BMW","BMW MOTORCYCLE"]), 'Vehicle_Make2'] = "07.BMW"
AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["NAZA"]), 'Vehicle_Make2'] = "08.NAZA"
AUTO_NA.loc[AUTO_NA.Vehicle_Make.isin(["MITSUBISHI"]), 'Vehicle_Make2'] = "09.MITSUBISHI"
AUTO_NA.Vehicle_Make2.fillna(-9999, inplace=True)
AUTO_NA.loc[AUTO_NA.Vehicle_Make2.isin([-9999]),'Vehicle_Make2'] = "10.Others";

AUTO_NA.loc[(AUTO_NA.Goods_Status == "N") & (AUTO_NA.Vehicle_Make.isin(["PERODUA","PROTON"])), 'VEHICLE_DETAIL']="01.NEW-NATIONAL"
AUTO_NA.loc[(AUTO_NA.Goods_Status == "N") & ~(AUTO_NA.Vehicle_Make.isin(["PERODUA","PROTON"])), 'VEHICLE_DETAIL']="02.NEW-NON NATIONAL"
AUTO_NA.loc[(AUTO_NA.Goods_Status == "U") & (AUTO_NA.Vehicle_Make.isin(["PERODUA","PROTON"])), 'VEHICLE_DETAIL']="03.USED-NATIONAL"
AUTO_NA.loc[(AUTO_NA.Goods_Status == "U") & ~(AUTO_NA.Vehicle_Make.isin(["PERODUA","PROTON"])), 'VEHICLE_DETAIL']="04.USED-NON NATIONAL"
AUTO_NA.loc[(AUTO_NA.Goods_Status == "R") & (AUTO_NA.Vehicle_Make.isin(["PERODUA","PROTON"])), 'VEHICLE_DETAIL']="05.RECOND-NATIONAL"
AUTO_NA.loc[(AUTO_NA.Goods_Status == "R" )& ~(AUTO_NA.Vehicle_Make.isin(["PERODUA","PROTON"])), 'VEHICLE_DETAIL']="06.RECOND-NON NATIONAL"
AUTO_NA.VEHICLE_DETAIL.fillna(-9999, inplace=True)
AUTO_NA.loc[AUTO_NA.VEHICLE_DETAIL.isin([-9999]),'VEHICLE_DETAIL']="07.NOT POPULATED"

AUTO_NA.Application_No = AUTO_NA.Application_No.astype(str)
AUTO_NA.Application_No = AUTO_NA.Application_No.str.upper()
AUTO_NA.Application_No = AUTO_NA.Application_No.str.strip()

application.Aa_number = application.Aa_number.astype(str)
application.Aa_number = application.Aa_number.str.upper()
application.Aa_number = application.Aa_number.str.strip()

AUTO_NA2 = AUTO_NA.merge(application[['Aa_number','Repayment_capacity']].rename(columns={'Aa_number':'Application_No'}), on="Application_No", how = 'left')

AUTO_NA2.Repayment_capacity.fillna(-9999, inplace=True)

def B_DSR(Repayment_capacity):
    if Repayment_capacity >= 0 and Repayment_capacity < 50:
        return "01.<50"
    elif Repayment_capacity >= 50 and Repayment_capacity < 70:
        return "02.50-<70"
    elif Repayment_capacity >= 50 and Repayment_capacity < 90:
        return "03.70-<90"
    elif Repayment_capacity >= 50 and Repayment_capacity < 100:
        return "04.90-<100"
    elif Repayment_capacity >= 100:
        return "05.>=100"
    else:
        return "06.NA"
AUTO_NA2['B_DSR'] = AUTO_NA2.Repayment_capacity.apply(B_DSR)

AUTO_NA2['NOA'] = 1

AUTO_NA2['Loan_Amount_MIL'] = AUTO_NA.Loan_Amount/1000000

row = ['Loan_Amount_MIL','NOA']

RISK_GRADE = AUTO_NA2.groupby(['RISK_GRADE'])[row].sum().reset_index()
RISK_GRADE1 = AUTO_NA2.groupby(['B_INCOME'])[row].sum().reset_index()
RISK_GRADE2 = AUTO_NA2.groupby(['Gender'])[row].sum().reset_index()
RISK_GRADE3 = AUTO_NA2.groupby(['B_DSR'])[row].sum().reset_index()
RISK_GRADE4 = AUTO_NA2.groupby(['B_MOF'])[row].sum().reset_index()
RISK_GRADE5 = AUTO_NA2.groupby(['B_TENURE'])[row].sum().reset_index()
RISK_GRADE6 = AUTO_NA2.groupby(['Region'])[row].sum().reset_index()
RISK_GRADE7 = AUTO_NA2.groupby(['Vehicle_Make2'])[row].sum().reset_index()
RISK_GRADE8 = AUTO_NA2.groupby(['VEHICLE_DETAIL'])[row].sum().reset_index()

writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\04_AITAB_Financing\04_Dataset\\'+str(date)[:6]+'\AITAB_NA'+str(date)+'(python).xlsx',engine='xlsxwriter')

RISK_GRADE.to_excel(writer2, sheet_name='RISK_GRADE', index = False)
RISK_GRADE1.to_excel(writer2, sheet_name='B_INCOME', index = False)
RISK_GRADE2.to_excel(writer2, sheet_name='Gender', index = False)
RISK_GRADE3.to_excel(writer2, sheet_name='B_DSR', index = False)
RISK_GRADE4.to_excel(writer2, sheet_name='B_MOF', index = False)
RISK_GRADE5.to_excel(writer2, sheet_name='B_TENURE', index = False)
RISK_GRADE6.to_excel(writer2, sheet_name='Region', index = False)
RISK_GRADE7.to_excel(writer2, sheet_name='Vehicle_Make2', index = False)
RISK_GRADE8.to_excel(writer2, sheet_name='VEHICLE_DETAIL', index = False)

writer2.save()